## Dataset: CESNET + Ucdavis-icdm19
#### Author: Jakub Čoček (xcocek00)

| Augmentations | Filters | Model | Validation | Fine-tuning | Testing |
|---------------|---------|----------|------------|----------------|---------|
| Change in Inter-Arrival Time (IAT) | none | trained on CESNET | n/a | Ucdavis - train | Ucdavis - script |


In [ ]:
# -- IMPORTS --

import os

import csv
import sys

# torch imports
import torch
import torch.nn as nn
from torcheval.metrics import MulticlassAccuracy
from torchdata.datapipes.iter import FileLister

# flowmind imports
os.chdir('/workplace/flowmind/')
from flowmind.contrastive import NTXentLoss

# livelossplot imports
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

# others
from functools import partial

# common imports
os.chdir('/workplace/xcocek00/common/')
from dataloader import create_flowpic_dataloader

os.chdir('/workplace/xcocek00/common/')
from nn import CNN, MLP

# sets csv limit
csv.field_size_limit(sys.maxsize)

# sets the device to use gpu if available, if not, use cpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
# -- custom remap

def remap_label(t: tuple[torch.Tensor, torch.Tensor, str], labels_d: dict[str, int]) -> tuple[torch.Tensor, torch.Tensor, int]:
    return t[0], t[1], labels_d[t[2]]

labels_ucdavis = [
    "google-doc",
    "google-drive",
    "google-music",
    "google-search",
    "youtube",
]

labels_ucdavis_d = {label: i for i, label in enumerate(labels_ucdavis)}

remap_label_ucdavis = partial(remap_label, labels_d=labels_ucdavis_d)


In [ ]:
# -- DATA LOADERS --
# dataloaders yield tuple (flowpic1, flowpic2, label)
# using only 15 seconds of the flow

# dataloader used for fine-tuning
dl_fine_t = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/train.csv",    
    batch_size=32,
    meta_key="app",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
    dp_transform=lambda dp: dp.map(remap_label_ucdavis).in_memory_cache(),
)

# dataloader used for testing
dl_test = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/script.csv",
    batch_size=32,
    meta_key="app",
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
    dp_transform=lambda dp: dp.map(remap_label_ucdavis).in_memory_cache(),
)


In [ ]:
# -- training loops--

def classification(CNN_model, MLP_model, dataloader, log, loss_fn, accuracy, optimizer):
    '''
    Classification loop

    Args:
        CNN_model: CNN model 
        MLP_model: MLP model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        log: for log collection
        loss_fn: chosen loss function
        accuracy: chosen model accuracy
        optimizer: chosen optimizer

    Returns:
        classification loss 
    '''

    # freeze the encoder
    for param in CNN_model.encoder.parameters():
        param.requires_grad = False

    # remove projection head
    CNN_model.projection = nn.Sequential()
    
    # set models
    CNN_model.eval()
    MLP_model.train()

    classification_loss = 0.0
    batches = 0

    for flowpic, _, label in dataloader:
        optimizer.zero_grad()

        flowpic, label = flowpic.to(device), label.to(device)

        with torch.no_grad():
            embeddings = CNN_model.encoder(flowpic)
        
        y_pred = MLP_model(embeddings)

        loss = loss_fn(y_pred, label)
        loss.backward()
        optimizer.step()

        classification_loss += loss.item()
        batches += 1
        
        accuracy.update(y_pred, label)
    
    classification_loss /= batches

    log["classification_loss"] = classification_loss
    log["accuracy"] = accuracy.compute()
        
    accuracy.reset()

    return classification_loss

def testing(CNN_model, MLP_model, dataloader, log, loss_fn, accuracy):
    '''
    Testing loop

    Args:
        CNN_model: CNN model 
        MLP_model: MLP model
        dataloader: pytorch dataloader created by create_flowpic_dataloader function
        log: for log collection
        loss_fn: chosen loss function
        accuracy: chosen model accuracy
        optimizer: chosen optimizer

    Returns:
        training loss 
    '''

    # freeze the encoder
    for param in CNN_model.encoder.parameters():
        param.requires_grad = False
    
    # set models
    CNN_model.eval()
    MLP_model.eval()

    test_loss = 0.0
    batches = 0

    with torch.no_grad():
        for flowpic_t, _, label in dataloader:
            flowpic_t, label = flowpic_t.to(device), label.to(device)

            embeddings = CNN_model.encoder(flowpic_t)
            y_pred = MLP_model(embeddings)

            loss = loss_fn(y_pred, label)
            test_loss += loss.item()
            batches += 1

            accuracy.update(y_pred, label)
    
        test_loss /= batches
    
        log["model_loss"] = test_loss
        log["model_accuracy"] = accuracy.compute()
            
        accuracy.reset()

    return test_loss



In [ ]:
# -- training linear classifier --

def train_classifier(cnn_model, mlp_model, dl_class, mlp_loss_fn, mlp_accuracy, mlp_optimizer, dl_test, mlp_liveloss):
    # set params for early stopping
    best_loss = float('inf')
    no_improvement = 0
    min_improvement = 0.001
    patience = 5
    best_acc = float('-inf')
    f_acc = 0

    for epoch in range(20):
        log = {}
        classification_loss = classification(cnn_model, mlp_model, dl_class, log, mlp_loss_fn, mlp_accuracy, mlp_optimizer)   
        test = testing(cnn_model, mlp_model, dl_test, log, mlp_loss_fn, mlp_accuracy)   

        mlp_liveloss.update(log)
        mlp_liveloss.send()

        if best_acc < log['model_accuracy']:
            best_acc = log['model_accuracy']

        # early stopping
        if best_loss - classification_loss > min_improvement:
            best_loss = classification_loss
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= patience:
            print("Stopping ... epoch ", epoch)
            break
            
    f_acc = log['model_accuracy']
    return f_acc, best_acc
    

In [ ]:
classes = 5

# loss functions
contrastive_loss_fn = NTXentLoss(temperature=0.07)
mlp_loss_fn = nn.CrossEntropyLoss()

final_acc_runs = []
best_acc_runs = []
for i in range(5):
    # load CNN model
    cnn_model = CNN().to(device)
    cnn_model.load_state_dict(torch.load("/workplace/xcocek00/models/cnn_model.pth"))

    # Linear classifier
    mlp_model = MLP(classes).to(device)
    mlp_optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.01)
    mlp_liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6, 2))])
    mlp_accuracy = MulticlassAccuracy()
    
    # training loops
    f_acc, b_acc = train_classifier(cnn_model, mlp_model, dl_fine_t, mlp_loss_fn, mlp_accuracy, mlp_optimizer, dl_test, mlp_liveloss)

    final_acc_runs.append(f_acc)
    best_acc_runs.append(b_acc)

    # print accuracy
    print(f"Run {i}: final accuracy: {f_acc}")
    print(f"Run {i}: best accuracy: {b_acc}")

# calculate mean
f_acc_p = torch.stack(final_acc_runs)
b_acc_p = torch.stack(best_acc_runs)

mean_f_acc = f_acc_p.mean().item()
std_f_acc = f_acc_p.std(unbiased=True).item()

mean_b_acc = b_acc_p.mean().item()
std_b_acc = b_acc_p.std(unbiased=True).item()

print(f"Final accuracy: {mean_f_acc:.4f} ± {std_f_acc:.4f}")
print(f"Best accuracy: {mean_b_acc:.4f} ± {std_b_acc:.4f}")